In [2]:
import os, sys
from tabulate import tabulate
from tqdm import tqdm
import pandas as pd
from z3 import set_param

os.chdir("/home/yizhou7/mariposa")

from utils.system_utils import log_info
from debugger.trace_analyzer import shorten_qid
from debugger3 import Debugger3

UNSTABLE_10_SECS = {
    "data/projs/flowcert/base.z3/permission.14.smt2",
    "data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v__LikesBetree.3.smt2",
    "data/projs/splinterdb/base.z3/allocation_layer__LikesBetree_v.18.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_eventually_always_no_delete_resource_request_msg_in_flight.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__validation.3.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_from_after_get_stateful_set_step_to_after_update_stateful_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof.2.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__validation.3.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__zookeeper_api__lemma_zk_request_implies_step_helper.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__liveness__stateful_set_match.2.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.6.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_every_resource_create_request_implies_at_after_create_resource_step.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__liveness__daemon_set_match__lemma_from_after_get_daemon_set_step_to_after_update_daemon_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_update_resource_step.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match__lemma_from_after_get_resource_step_to_after_create_resource_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__resource_match.1.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__proof__lemma_eventually_always_object_in_response_at_after_create_resource_step_is_same_as_etcd.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__liveness__stateful_set_match__lemma_stateful_set_state_matches_at_after_update_stateful_set_step.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-rabbitmq-smt-rabbitmq_controller__proof__helper_invariants__validation__lemma_always_stateful_set_in_create_request_msg_satisfies_unchangeable.smt2",
    "data/projs/anvil/base.z3/zookeeper-smt-zookeeper_controller__proof__helper_invariants__proof__lemma_always_no_update_status_request_msg_in_flight_of_except_stateful_set.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit_config__proof__liveness__resource_match.5.smt2",
    "data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init.2.smt2",
    "data/projs/verismo.dedup/base.z3/security__monitor.21.smt2",
    "data/projs/verismo.dedup/base.z3/vbox__vbox.5.smt2",
    "data/projs/verismo.dedup/base.z3/arch__ptram__ptram_p2.smt2",
    "data/projs/verismo.dedup/base.z3/bsp__ap.3.smt2",
    "data/projs/verismo.dedup/base.z3/snp__cpuid.17.smt2",
    "data/projs/verismo.dedup/base.z3/snp__ghcb__proto_page.141.smt2",
    "data/projs/verismo.dedup/base.z3/pgtable_e__pte.20.smt2",
    "data/projs/verismo.dedup/base.z3/vbox__vbox.4.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init_alloc.1.smt2",
    "data/projs/verismo.dedup/base.z3/allocator__linkedlist.10.smt2",
    "data/projs/verismo.dedup/base.z3/pgtable_e__pte.18.smt2",
    "data/projs/verismo.dedup/base.z3/boot__init__e820_init.4.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.13.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__span_queue_delete.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.5.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-commit_segment.1.smt2",
    "data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.3.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.17.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.12.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.16.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-config.35.smt2",
    "data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.5.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__logimpl_v.6.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.5.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__start_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.14.smt2",
    "data/projs/vsystemsnew/base.z3/page-table-smt-impl_u__l2_refinement.4.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg.69.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.17.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__setup_v.6.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.11.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_os_alloc.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.7.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.4.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-page_organization__PageOrg__impl_%4__merge_with_before_ll_inv_valid_unused.smt2",
    "data/projs/vsystemsnew/base.z3/mimalloc-smt-segment__segment_span_free_coalesce_before.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.15.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.9.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.2.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.13.smt2",
    "data/projs/vsystemsnew/base.z3/noderep-smt-spec__cyclicbuffer.3.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__inv_v.10.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-multilog__inv_v.14.smt2",
    "data/projs/vsystemsnew/base.z3/verified-storage-smt-verified-storage-smt-storage-node-log__layout_v.30.smt2",
    "data/projs/atmosphere/base.z3/kernel__create_and_share_pages.5.smt2",
}

# def load_scores():
#     scores = dict()

#     for line in open("/home/yizhou7/mariposa/doc/scores.csv", "r"):
#         parts = line.strip().split(",")
#         query = parts[0]
#         scores[query] = [int(s) for s in parts[1:]]
        
#     return scores

# scores = load_scores()

# print("total unstable:", len(UNSTABLE_10_SECS))
# print("no proofs:", len(UNSTABLE_10_SECS - set(scores.keys())))

# no_fixes = set()
# bad_scores = set()

# for query, score in scores.items():
#     if score[0] == 0:
#         no_fixes.add(query)
#     elif all([s == 0 for s in score[1:]]):
#         bad_scores.add(query)

# print("no fixes:", len(no_fixes))
# print("bad scores:", len(bad_scores))

# for query in sorted(bad_scores):
#     print(query, scores[query][0])

ImportError: cannot import name 'InstError' from 'proof_builder' (/home/yizhou7/mariposa/src/proof_builder.py)

In [2]:
from debugger.edit_info import EditAction

dbg = Debugger3("data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2")

[INFO] [init] analyzing data/projs/anvil/base.z3/fluent-smt-fluent_controller__fluentbit__proof__helper_invariants__proof.9.smt2 in dbg/199ba4594c 
[INFO] proof QI graph is acyclic 
[INFO] [graph] found dbg/199ba4594c/graphs/reseed.15196461297475196408.txt 
[INFO] [stats] found dbg/199ba4594c/stats/reseed.15196461297475196408.txt 
[WARN] [report] already exists: dbg/199ba4594c/report.csv 


In [8]:
from debugger.edit_info import EditInfo
import random
from base.solver import RCode

def debugger_erase_search(dbg: Debugger3):
    actions = dbg.differ.actions
    erasable = []

    for qid, action in actions.items():
        if action == EditAction.ERASE:
            erasable.append(qid)

    print("total erasable:", len(erasable))
    ei: EditInfo = dbg.test_edit(set(erasable))
    print("ideal:", ei.rcode, ei.time)

    current = random.sample(erasable, 1)
    ei: EditInfo = dbg.test_edit(set(current))
    print("current:", ei.rcode, ei.time)

    if ei.rcode != RCode.UNSAT:
        print("bad: ", current)

    # print("current:", current)
    # print(ei.rcode, ei.time)

debugger_erase_search(dbg)


total erasable: 454
ideal: unsat 0.42
[INFO] [run] dbg/199ba4594c/edits/90a08e1b.smt2 unsat  3.35 
current: unsat 3.35


In [ ]:
import sexpdata as sexp
proof_file = "dbg/815f69b161/proofs/shuffle.13565831226465156427.proof"
with open(proof_file) as f:
    data = sexp.load(f)


<class 'list'>


In [12]:
PROOF_RULES = {
    "true", 
    "der", 
    "asserted", 
    "quant-inst", 
    "goal", 
    "hypothesis", 
    "mp", 
    "lemma", 
    "unit-resolution", 
    "refl", 
    "iff-true", 
    "symm", 
    "iff-false", 
    "trans", 
    "commutativity", 
    "monotonicity", 
    "def-axiom", 
    "quant-intro", 
    "def-intro", 
    "distributivity", 
    "apply-def", 
    "and-elim", 
    "iff", 
    "not-or-elim", 
    "nnf-pos", 
    "rewrite", 
    "nnf-neg", 
    "pull-quant", 
    "sk", 
    "push-quant", 
    "elim-unused-vars", 
    "th-lemma", 
}

Z3_BUILTIN = {
    "and",
    "or",
    "not",
    "=>",
    "ite",
    "=",
    "+",
    "-",
    "*",
}

def get_symbol(data):
    assert isinstance(data, sexp.Symbol)
    return data.value()

def parse_var_bindings(data):
    assert isinstance(data, list)
    bindings = dict()
    for binding in data:
        assert isinstance(binding, list)
        assert len(binding) == 2
        var = get_symbol(binding[0])
        val = parse(binding[1])
        bindings[var] = val

def parse(data):
    if isinstance(data, sexp.Symbol):
        return data.value()
    assert isinstance(data, list)
    app = get_symbol(data[0])

    if app == "let":
        assert len(data) == 3
        bindings = parse_var_bindings(data[1])
        return parse(data[2])
    else:
        print("TODO", app)

class ProofParser:
    def __init__(self, data):
        self.data = data
        self.unknown_symbols = set()
        self.sanity_check(data)

        for symbol in self.unknown_symbols:
            print(symbol)

    def check_symbol(self, data):
        assert isinstance(data, sexp.Symbol)
        symbol = data.value()
        if symbol not in PROOF_RULES and symbol not in Z3_BUILTIN:
            self.unknown_symbols.add(data.value())
        return

    def sanity_check(self, data):
        if isinstance(data, sexp.Symbol):
            self.check_symbol(data)
            return
        if isinstance(data, list):
            for d in data:
                self.sanity_check(d)
            return
        if isinstance(data, int):
            return

        print("TODO", type(data))

parser = ProofParser(data)

a!1511
a!14
a!5339
a!2220
prelude_eucmod
a!1809
fuel%vstd!slice.group_slice_axioms.
a!4956
a!5258
a!1810
a!2652
a!154
a!3686
a!3545
a!2922
a!5448
a!1573
a!3078
a!2594
a!3889
a!5304
a!701
a!2306
a!2180
a!2944
a!2237
a!5329
a!3213
a!6789
a!4773
a!3056
a!1879
a!6625
a!6591
a!5916
a!5918
a!5010
a!935
a!3254
a!1086
a!1886
fuel%vstd!set_lib.group_set_lib_axioms.
a!663
core!option.Option.
a!3248
a!1620
a!5396
a!805
a!6278
a!4699
a!2725
a!2669
a!2558
a!1825
a!5334
a!5700
a!6742
a!3705
a!980
a!5459
a!824
a!999
a!6575
lib!pmem.pmcopy_t.SpecPmSized.spec_size_of.?
a!5547
a!4967
-
a!5190
a!1580
a!3785
a!622
a!4582
a!6047
a!3852
a!1702
a!2579
a!3813
a!5858
a!6759
a!1225
a!3680
a!5849
a!2361
a!3010
a!4988
a!106
a!2305
a!1165
a!1686
a!3063
a!5145
a!447
a!5353
a!3800
a!2977
a!6796
a!1219
a!6160
a!5773
a!5266
a!2383
fuel%lib!pmem.pmcopy_t.impl&%27.spec_size_of.
a!3088
a!2846
a!6294
a!3701
a!3730
a!1492
core!option.Option./Some/?0
a!6815
a!5060
a!6417
a!4763
a!2798
a!4207
a!3666
a!4404
a!6318
a!33
a!5888